In [12]:
import numpy as np
import os
import cv2
import random

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [13]:
data_path = 'sdc/train'
train_data = []
val_data = []

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    images = os.listdir(folder_path)
    num_train = int(0.8 * len(images))
    train_images = random.sample(images, num_train)
    val_images = list(set(images) - set(train_images))
    
    for img_file in train_images:
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        train_data.append((img, folder))
    
    for img_file in val_images:
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        val_data.append((img, folder))


In [14]:
X_train, y_train = zip(*train_data)
X_val, y_val = zip(*val_data)

X_train = preprocess_input(np.array(X_train))
X_val = preprocess_input(np.array(X_val))

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(le.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)


In [15]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)


In [16]:
history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop, reduce_lr]
)


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 1011s 2s/step - accuracy: 0.2584 - loss: 2.4752 - val_accuracy: 0.3914 - val_loss: 1.8993 - learning_rate: 0.0010
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 989s 2s/step - accuracy: 0.3882 - loss: 1.9153 - val_accuracy: 0.4460 - val_loss: 1.7643 - learning_rate: 0.0010
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 987s 2s/step - accuracy: 0.4410 - loss: 1.7517 - val_accuracy: 0.4864 - val_loss: 1.6749 - learning_rate: 0.0010
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 993s 2s/step - accuracy: 0.4782 - loss: 1.6452 - val_accuracy: 0.5024 - val_loss: 1.5922 - learning_rate: 0.0010
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 1006s 2s/step - accuracy: 0.4980 - loss: 1.5710 - val_accuracy: 0.5269 - val_loss: 1.5215 - learning_rate: 0.0010
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 1061s 2s/step - accuracy: 0.5160 - loss: 1.5003 - val_accuracy: 0.5495 - val_loss: 1.4743 - learning_rate: 0.0010
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2483s 5s/step - accuracy: 0.5437 - loss

KeyboardInterrupt: 

In [ ]:
train_loss, train_acc = model.evaluate(train_ds)
val_loss, val_acc = model.evaluate(val_ds)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")

1313/1313 ━━━━━━━━━━━━━━━━━━━━ 391s 298ms/step - accuracy: 0.9732 - loss: 0.1207
170/170 ━━━━━━━━━━━━━━━━━━━━ 56s 328ms/step - accuracy: 0.6406 - loss: 1.4875
Train Accuracy: 0.9761
Validation Accuracy: 0.6368


In [ ]:
model.save("DenseNet201.h5")